In [28]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict,Annotated,Optional
from pydantic import Field, BaseModel
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [29]:
class Review(TypedDict):
  # Annotate used to add descriptions to the fields
  summary: Annotated[str,"A brief summary of the review"]
  sentiment: Annotated[str,"The overall sentiment of the review, e.g., positive, negative, neutral"]

structured_model = model.with_structured_output(Review)

result = structured_model. invoke("""The hardware is great, but the software feels bloated.
There are too many pre-installed apps that I can't remove. Also, the UI looks outdated
compared to other brands. Hoping for a software update to fix this.""")

result

{'summary': 'The hardware is great, but the software is bloated with unremovable pre-installed apps and an outdated UI. The user hopes for a software update.',
 'sentiment': 'negative'}

In [ ]:
class ReviewModel(BaseModel):
  summary:str = Field(description="A brief summary of the review")
  sentiment:str = Field(default=None, description="The overall sentiment of the review, e.g., positive, negative")
  suggestions: Optional[str] = Field(default=None, description="Suggestions for improvement mentioned in the review")
  
structured_model = model.with_structured_output(ReviewModel)
result = structured_model.invoke("The product was okay, but could be improved in several areas.")
result

ReviewModel(summary='The product was okay, but could be improved in several areas.', sentiment='Neutral')

In [24]:
json_schema = {
  "type": "object",
  "properties": {
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "description": "The overall sentiment of the review, e.g., positive, negative"
    },
    "suggestions": {
      "type": "string",
      "description": "Suggestions for improvement mentioned in the review"
    }
  },
  "required": ["summary", "sentiment"]
}

structured_model = model.with_structured_output(json_schema)
result = structured_model.invoke("The product was okay, but could be improved in several areas.")
result

{'summary': 'The product was okay, but needs improvement.',
 'sentiment': 'neutral',
 'suggestions': 'Improvements needed in several areas.'}

In [30]:
# Enum
from enum import Enum

class Sentiment(str, Enum):  # Enum as Pydantic string subclass
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"

class ReviewEnumModel(BaseModel):
    summary: str = Field(description="A brief summary of the review")
    sentiment: Sentiment = Field(description="The overall sentiment of the review")
    
    
structured_model = model.with_structured_output(ReviewEnumModel)
result = structured_model.invoke("The product was okay, but could be improved in several areas.")
result

ReviewEnumModel(summary='The product was satisfactory but had room for improvement.', sentiment=<Sentiment.NEUTRAL: 'neutral'>)

In [32]:
result.sentiment.value

'neutral'